## Bandwith-Evaluation (MESH)

### 1. Install dependencies

In [1]:
%%capture
!pip install ipykernel;
!pip install matplotlib;
!pip install seaborn;
!pip install folium;
!pip install plotly;

import pandas as pd;
import geopandas as gpd;
import folium;
import os;
import seaborn as sns;
import plotly.figure_factory as ff;

### 2. Load testtraces

In [24]:
folder = 'results'

testtraces = {}
for dir in os.listdir(folder):
      testtraces[dir] = []
      try:
          for file in os.listdir(os.path.join(folder, dir)):
              if "pcap" not in file:
                  file_path = os.path.join(folder, dir, file)
                  df = pd.read_csv(file_path)
                  testtraces[dir].append(df)
      except Exception as e:
          print(f"Failed to read {file_path}: {e}")


In [30]:
testtraces['bandwithtest_09072924_mesh_wlan-b'][0].head()

,Unnamed: 0,Time,Latitude,Longitude,Bitrate,Latency,DISTANCE_CENTER,DISTANCE_AP_RUESTHALLE,DISTANCE_AP_GARAGE
0,0,11:38:58.500000,52.315857,10.561806,88.7,1.77,2.322884,40.281297,41.945280
1,1,11:38:59.500000,52.315858,10.561806,94.1,24.40,2.201237,40.242462,41.983369
2,2,11:39:00.500000,52.315859,10.561807,73.4,26.60,2.077964,40.236075,41.989827
3,3,11:39:01.500000,52.315862,10.561814,83.9,27.20,2.071589,40.645461,41.586773
4,4,11:39:02.500000,52.315870,10.561840,94.4,24.80,3.261614,42.253782,40.044156


### 3.1. Plotting of bandwith-results depending on distance to center:

In [48]:
# Concatenate all dataframes to plot on a single figure
all_dfs = []
for testname, list_of_dfs in testtraces.items():
    all_dfs.extend(list_of_dfs)

combined_df = pd.concat(all_dfs, ignore_index=True)

# Outlier removal using IQR
def remove_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

combined_df_filtered = combined_df.copy()
combined_df_filtered = remove_outliers_iqr(combined_df_filtered, 'Bitrate')
combined_df_filtered = remove_outliers_iqr(combined_df_filtered, 'DISTANCE_CENTER')

In [62]:
# plottling
scatter_fig = px.scatter(
    combined_df_filtered,
    x='DISTANCE_CENTER',
    y='Bitrate',
    title='Bitrate vs. Distance to Center with Trendline (Outliers Removed)',
    labels={'DISTANCE_CENTER': 'Distance to Center [m]', 'Bitrate': 'Bitrate [MBit/s]'},
    trendline='lowess',
)

histogram_fig = go.Figure(go.Histogram2dContour(
    x=combined_df_filtered['DISTANCE_CENTER'],
    y=combined_df_filtered['Bitrate'],
    colorscale='Viridis',
    colorbar=dict(title='Density'),
    contours=dict(coloring='heatmap'),
))

# Create subplots
fig = make_subplots(rows=1, cols=2, subplot_titles=('Bitrate vs. Distance to Center with Trendline', 'Spread of Bitrate vs. Distance to Center (Heatmap)'))

for trace in scatter_fig['data']:
    fig.add_trace(trace, row=1, col=1)

for trace in histogram_fig['data']:
     fig.add_trace(trace, row=1, col=2)


fig.update_layout(
    title_text='Bitrate Visualizations',
    template='none'
)

fig.update_xaxes(title_text='Distance to Center [m]', row=1, col=1)
fig.update_yaxes(title_text='Bitrate [MBit/s]', row=1, col=1)
fig.update_xaxes(title_text='Distance to Center [m]', row=1, col=2)
fig.update_yaxes(title_text='Bitrate [MBit/s]', row=1, col=2)

fig.show()

### 3.2. Spread of Bitrate vs. Distance to Center

### 3.4. Plotting of Latency results depending on distance to center:

In [ ]:
# Concatenate dataframes with 'Latency' column
all_dfs_latency = []
for testname, list_of_dfs in testtraces.items():
    for df in list_of_dfs:
        if 'Latency' in df.columns:
            all_dfs_latency.append(df)

combined_df_latency = pd.concat(all_dfs_latency, ignore_index=True)

# Outlier removal using IQR for Latency and DISTANCE_CENTER
def remove_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

combined_df_latency_filtered = combined_df_latency.copy()
combined_df_latency_filtered = remove_outliers_iqr(combined_df_latency_filtered, 'Latency')
combined_df_latency_filtered = remove_outliers_iqr(combined_df_latency_filtered, 'DISTANCE_CENTER')

In [63]:
# Create the scatter plot with trendline
scatter_fig_latency = px.scatter(
    combined_df_latency_filtered,
    x='DISTANCE_CENTER',
    y='Latency',
    title='Latency vs. Distance to Center with Trendline (Outliers Removed)',
    labels={'DISTANCE_CENTER': 'Distance to Center [m]', 'Latency': 'Latency [ms]'},
    trendline='lowess' # Add a LOESS trendline
)

# Create the 2D histogram
histogram_fig_latency = go.Figure(go.Histogram2dContour(
    x=combined_df_latency_filtered['DISTANCE_CENTER'],
    y=combined_df_latency_filtered['Latency'],
    colorscale='Viridis',
    colorbar=dict(title='Density'),
    contours=dict(coloring='heatmap'),
))

# Create subplots
fig_latency = make_subplots(rows=1, cols=2, subplot_titles=('Latency vs. Distance to Center with Trendline (Outliers Removed)', 'Spread of Latency vs. Distance to Center (2D Histogram, Outliers Removed)'))

# Add the scatter plot trace and layout from scatter_fig_latency
for trace in scatter_fig_latency['data']:
    fig_latency.add_trace(trace, row=1, col=1)

# Add the histogram trace from histogram_fig_latency
for trace in histogram_fig_latency['data']:
     fig_latency.add_trace(trace, row=1, col=2)


# Update layout for titles and axis labels
fig_latency.update_layout(
    title_text='Latency Visualizations (Outliers Removed)',
    showlegend=False # Hide legend if not needed
)

fig_latency.update_xaxes(title_text='Distance to Center [m]', row=1, col=1)
fig_latency.update_yaxes(title_text='Latency [ms]', row=1, col=1)
fig_latency.update_xaxes(title_text='Distance to Center [m]', row=1, col=2)
fig_latency.update_yaxes(title_text='Latency [ms]', row=1, col=2)


fig_latency.show()

### 4. Bandwith and Latency on geomap

In [79]:
# Concatenate all dataframes for plotting
all_dfs_map = []
for testname, list_of_dfs in testtraces.items():
    for df in list_of_dfs:
        all_dfs_map.append(df)

combined_df_map = pd.concat(all_dfs_map, ignore_index=True)

# Outlier removal using IQR
def remove_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

combined_df_map_filtered = combined_df_map.copy()
combined_df_map_filtered = remove_outliers_iqr(combined_df_map_filtered, 'Bitrate')
combined_df_map_filtered = remove_outliers_iqr(combined_df_map_filtered, 'DISTANCE_CENTER')

# Drop rows with NaN in Latency column for plotting size and color on the Latency map
if 'Latency' in combined_df_map_filtered.columns and not combined_df_map_filtered['Latency'].isnull().all():
    combined_df_map_filtered = remove_outliers_iqr(combined_df_map_filtered, 'Latency')

combined_df_map_latency_filtered = combined_df_map_filtered.dropna(subset=['Latency']).copy()


# plotting
fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'mapbox'}, {'type': 'mapbox'}]],
                    subplot_titles=('Bitrate Map (Outliers Removed)', 'Latency Map (Outliers Removed)'))

# Bitrate
fig.add_trace(go.Scattermapbox(
    lat=combined_df_map_filtered["Latitude"],
    lon=combined_df_map_filtered["Longitude"],
    mode='markers',
    marker=go.scattermapbox.Marker(
        size=16, # Increased marker size
        color=combined_df_map_filtered["Bitrate"],
        colorscale='viridis',
        colorbar=dict(title='Bitrate [MBit/s]', x=0.45) # Adjust x position
    ),
    text=combined_df_map_filtered["Bitrate"].apply(lambda x: f'Bitrate: {x:.2f}'),
    name='Bitrate'
), row=1, col=1)

# Latency
green_to_red = [(0, 'green'), (0.5, 'grey'), (1, 'red')]
if not combined_df_map_latency_filtered.empty: # Check if there's data after dropping NaNs
    fig.add_trace(go.Scattermapbox(
        lat=combined_df_map_latency_filtered["Latitude"],
        lon=combined_df_map_latency_filtered["Longitude"],
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=16, # Increased marker size
            color=combined_df_map_latency_filtered["Latency"],
            colorscale=green_to_red,
            colorbar=dict(title='Latency [ms]', x=1.0) # Adjust x position
        ),
        text=combined_df_map_latency_filtered["Latency"].apply(lambda x: f'Latency: {x:.2f}'),
        name='Latency'
    ), row=1, col=2)
else:
     print("No Latency data available after outlier removal for mapping.")


# Update layout for both mapbox subplots
fig.update_layout(
    mapbox1=dict(
        style="carto-positron",
        center=dict(lat=combined_df_map_filtered["Latitude"].mean(), lon=combined_df_map_filtered["Longitude"].mean()),
        zoom=16
    ),
    mapbox2=dict(
        style="carto-positron",
        center=dict(lat=combined_df_map_latency_filtered["Latitude"].mean(), lon=combined_df_map_latency_filtered["Longitude"].mean()),
        zoom=16
    ),
    showlegend=False,
)

fig.show()